Authors: Nicolas Legrand <nicolas.legrand@cfin.au.dk>

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from metadPy import dprime, criterion

# Generate reports and clean data

Individual reports were generated using the following scripts (requires access to the raw data).

In [2]:
#%%capture
#!python reports_HRD.py
#!python reports_HRD2.py

# Del 1 

In [3]:
datapath = os.path.join(
    os.path.abspath(os.path.join(os.getcwd(), os.pardir)),
    'data', 'raw', 'HRD')

outputpath = os.path.join(
    os.path.abspath(os.path.join(os.getcwd(), os.pardir)),
    'data')
session = 'Del1'
subjects = os.listdir(datapath)

After check the individual reports manually, it was decided to remove the following participants from the group analysis.

In [4]:
remove = ['sub_0036', 'sub_0041', 'sub_0054', 'sub_0067', 'sub_0069', 'sub_0078', 'sub_0084', 'sub_0086', 'sub_0139', 'sub_0143', 'sub_0173', 'sub_0234']

Merge dataframes

In [5]:
merged_df = pd.DataFrame([])  # Concatenated raw data
for sub in subjects:
    df = pd.read_csv(os.path.join(datapath, sub, 'filtered.txt'))
    df["Subject"] = sub
    df = df.rename(columns={'Estimation': 'Descision', 'EstimationRT': 'DecisionRT'})
    merged_df = merged_df.append(df, ignore_index=True)

# Remove particpants
for sub_rm in remove:
    merged_df = merged_df[merged_df.Subject != sub_rm]

# Save data frame
merged_df.to_csv(
    os.path.join(
        outputpath, 'Del1_merged.txt'),
    index=False)

# Del2

This script will import raw data from AUX and build a local database containing only essential files for further preprocessing. You can select where to store these file by changing `localPath`.

In [6]:
datapath = os.path.join(
    os.path.abspath(os.path.join(os.getcwd(), os.pardir)),
    'data', 'raw', 'HRD2')

outputpath = os.path.join(
    os.path.abspath(os.path.join(os.getcwd(), os.pardir)),
    'data')
session = 'Del2'
subjects = os.listdir(datapath)

In [7]:
remove = ['sub_0139']

Merge dataframes

In [8]:
merged_df = pd.DataFrame([])  # Concatenated raw data
for sub in subjects:
    df = pd.read_csv(os.path.join(datapath, sub, 'filtered.txt'))
    df["Subject"] = sub
    merged_df = merged_df.append(df, ignore_index=True)

# Remove particpants
for sub_rm in remove:
    merged_df = merged_df[merged_df.Subject != sub_rm]

# Save data frame
merged_df.to_csv(
    os.path.join(
        outputpath, 'Del2_merged.txt'),
    index=False)

# Summary dataframe

Extract summary statistics

In [9]:
def reversals(df):
    d = np.diff(df.Alpha.to_numpy())
    reversals = d[:-1] != d[1:]
    return np.median(df.Alpha.to_numpy()[1:-1][reversals])

In [10]:
pd.read_csv(os.path.join(outputpath, 'Del2_merged.txt'))

,TrialType,Condition,Modality,StairCond,Decision,DecisionRT,Confidence,ConfidenceRT,Alpha,listenBPM,...,EstimatedThreshold,EstimatedSlope,StartListening,StartDecision,ResponseMade,RatingStart,RatingEnds,endTrigger,HeartRateOutlier,Subject
0,psiCatchTrial,Less,Intero,psiCatchTrial,Less,1.933293,99.0,0.512461,-30.0,66.5,...,NaN,NaN,1.603353e+09,1.603353e+09,1.603353e+09,1.603353e+09,1.603353e+09,1.603353e+09,False,sub_0019
1,psi,Less,Extero,psi,Less,2.216429,59.0,1.632995,-0.5,78.0,...,22.805550,12.549457,1.603353e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,False,sub_0019
2,psiCatchTrial,More,Intero,psiCatchTrial,Less,3.099441,25.0,0.729331,10.0,67.0,...,NaN,NaN,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,False,sub_0019
3,psiCatchTrial,Less,Extero,psiCatchTrial,Less,1.449154,100.0,0.511938,-30.0,82.0,...,NaN,NaN,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,False,sub_0019
4,psi,Less,Intero,psi,Less,2.099316,41.0,1.243842,-0.5,65.5,...,22.805550,12.549457,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,1.603354e+09,False,sub_0019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22435,psi,More,Extero,psi,Less,3.931575,11.0,1.200633,0.5,80.5,...,3.443235,4.849067,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,False,sub_0255
22436,psi,More,Extero,psi,More,2.747645,54.0,0.815499,0.5,68.5,...,2.983697,5.166114,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,False,sub_0255
22437,psi,Less,Intero,psi,Less,3.248018,68.0,0.512670,-24.5,82.5,...,-11.002944,11.504530,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,False,sub_0255
22438,psi,More,Extero,psi,More,3.346473,81.0,0.629611,8.5,99.0,...,2.884844,5.040929,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,1.603376e+09,False,sub_0255


In [11]:
behavior_df = pd.DataFrame([])

for session in ['Del1', 'Del2']:

    group_df = pd.read_csv(
        os.path.join(outputpath, f'{session}_merged.txt'))
 
    for sub in group_df['Subject'].unique():

        taskDuration = (
            group_df.loc[group_df.Subject == sub, "StartListening"].to_numpy()[-1] -
                group_df.loc[group_df.Subject == sub, "StartListening"].to_numpy()[0]
            ) / 60

        for modality in ["Intero", "Extero"]:
            
            threshold, slope, decisionRT, confidenceRT, accuracy, confidence,\
                threshold_updown = None, None, None, None, None, None, None
            
            this_df = group_df[(group_df.Subject == sub) & (group_df.Modality == modality)]

            threshold, slope = (
                this_df[~this_df.EstimatedThreshold.isnull()].EstimatedThreshold.iloc[-1],
                this_df[~this_df.EstimatedSlope.isnull()].EstimatedSlope.iloc[-1],
            )
            decisionRT, confidenceRT = (
                this_df["DecisionRT"].median(),
                this_df.ConfidenceRT.median(),
            )
            accuracy, confidence = (
                this_df["ResponseCorrect"].mean() * 100,
                this_df["Confidence"].mean(),
            )

            # Threshold from up/down staircase
            threshold_updown = np.mean([
                reversals(this_df[this_df.StairCond=='low']),
                reversals(this_df[this_df.StairCond=='high'])
            ])

            # Ratio of staircase corruption
            ratio = ((this_df.Alpha - this_df.EstimatedThreshold).loc[-40:] > 0).mean()
            corruption_ratio = np.abs(ratio-0.5)*2

            behavior_df = behavior_df.append(
                {
                    "Subject": sub,
                    "Session": session,
                    "Modality": modality,
                    "Accuracy": accuracy,
                    "Confidence": confidence,
                    "Threshold": threshold,
                    "Threshold_UpDown": threshold_updown,
                    "Slope": slope,
                    "TaskDuration": taskDuration,
                    "DecisionRT": decisionRT,
                    "ConfidenceRT": confidenceRT,
                    "dPrime": d,
                    "Criterion": c,
                    "Corruption": corruption_ratio,
                },
                ignore_index=True,
            )

# Save data frame
behavior_df.to_csv(
    os.path.join(
        outputpath, 'behavior.txt'),
    index=False)

NameError: name 'd' is not defined

## Task descriptive stats

In [ ]:
behavior_df = pd.read_csv(os.path.join(outputpath, 'behavior.txt'))

In [ ]:
print(f'Number of participants: {len(behavior_df.Subject.unique())}')
del1 = len(behavior_df[behavior_df.Session=='Del1'].Subject.unique())
print(f'Del 1: {del1}')
del2 = len(behavior_df[behavior_df.Session=='Del2'].Subject.unique())
print(f'Del 1: {del2}')

In [ ]:
sns.set_context('talk')
plt.figure(figsize=(8, 5))
sns.histplot(data=behavior_df[behavior_df.Modality=='Intero'],
             x='TaskDuration', hue='Session')
plt.xlabel('Time (min)')
plt.ylabel('HRD')
sns.despine()

In [ ]:
for session in ['Del1', 'Del2']:
    print(f'Session: {session} - Task mean time: {round(behavior_df[behavior_df.Session == session].TaskDuration.mean(), 2)} min')
    print(f'Session: {session} - Task std time: {round(behavior_df[behavior_df.Session == session].TaskDuration.std(), 2)} min')
    print(f'Session: {session} - Task max time: {round(behavior_df[behavior_df.Session == session].TaskDuration.max(), 2)} min')
    print(f'Session: {session} - Task min time: {round(behavior_df[behavior_df.Session == session].TaskDuration.min(), 2)} min')